In [1]:
import ray
ray.init()

2023-03-01 22:09:47,502	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2023-03-01_22-09-40_571050_26127/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2023-03-01_22-09-40_571050_26127/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2023-03-01_22-09-40_571050_26127',
 'metrics_export_port': 57927,
 'node_id': '0dc723e30fd75288fc63da4fe1ec223b9269ca643e27efa14e63e769'}

In [7]:
import time

database = ["A", "B", "C", "D", "E", "F", "G"]

def retrieve(item):
    time.sleep(item / 10.)
    return item, database[item]

start = time.time()
data = [retrieve(item) for item in range(len(database))]
end = time.time()
print(end - start)

2.1255509853363037


In [11]:
database_object_ref = ray.put(database)

@ray.remote
def retrieve_task(item):
    database = ray.get(database_object_ref)
    time.sleep(item / 10.)
    return item, database[item]

start = time.time()
data_ref = [retrieve_task.remote(item) for item in range(len(database))]
data = ray.get(data_ref)
end = time.time()
print(end - start)

0.6156361103057861


In [15]:
@ray.remote
def follow_up_task(retrieve_result):
    original_item, _ = retrieve_result
    follow_up_result = retrieve(original_item + 1)
    return retrieve_result, follow_up_result

ref1 = retrieve_task.remote(0)
ref2 = follow_up_task.remote(ref1)

print(ray.get(ref2))

((0, 'A'), (1, 'B'))
